# Captain's Log

In [196]:
# Setup

# pip3 install urllib3 # Might not have worked. Hard to say
# pip install urllib3
# pip3 install requests



In [202]:
#
#   MODEL - data structures
#

# Score data structure
class Score:
    def __init__(self, total, item):
        self.total = total
        self.item = item
    def __repr__(self):
        return repr((self.total, self.item))

# SuperPoint data structure
class SuperPoint:
    def __init__(self, item):
        self.items = []
        self.items.append(item)
    def __repr__(self):
        return repr((self.items))



In [204]:
#
# UTILITY methods
#
from math import sqrt

# Get the distance between two points
def getDistance(loc1, loc2):
    dist = sqrt( (loc2['lon'] - loc1['lon'])**2 + (loc2['lat'] - loc1['lat'])**2 )
    return dist

# Are two points close enough
def closeEnough(loc1, loc2, limitDistance):
    thisDistance = getDistance(loc1, loc2)
    return thisDistance <= limitDistance

# Count the number of points within a distance of a point
def getScore(point, allPoints, limitDistance):
    result = 0
    for other in allPoints:
        if closeEnough(point, other, limitDistance):
            result += 1
    return result

# Convert anything to json
import json
def getJson(something):
    json_string = json.dumps(something)
    return json_string



# A super point is based off of an initial point
def createSuperPoint():
    point = sorted_scores[0].item
    sorted_scores.pop(0)
    sp = SuperPoint(point)
    i = 0
    while i < len(sorted_scores):
        other = sorted_scores[i].item
        if closeEnough(point, other, arbitaryDistanceLimit):
            sp.items.append(other)
            sorted_scores.pop(0)
            i -= 1
        i += 1
    return sp


# All subsequent points must also be checked for near-neighbors
def nourishSuperPoint(sp):
    for point in sp.items:
        i = 0
        while i < len(sorted_scores):
            other = sorted_scores[i].item
            if closeEnough(point, other, arbitaryDistanceLimit):
                sp.items.append(other)
                sorted_scores.pop(0)
            i += 1




In [205]:
#
# SETUP
#

import json, urllib.request, requests
from collections import namedtuple

def getLocations(url):
    r = requests.get(url)
    j = json.loads(r.text)
    return j

sample_locations = getLocations('https://geo-pg.captainjustin.space/transactions_rs_time')

# Decide on accuracy
arbitaryDistanceLimit = 0.0001
arbitaryDistanceLimit = 0.001




In [225]:
### Let's score each point by how many other points are within `arbitaryDistanceLimit`


# We'll keep scores here in an array here
scores = [] 


# Get the score for each location
for l in sample_locations:
    total = getScore(l, sample_locations, arbitaryDistanceLimit)
    scores.append(Score(total, l))

sorted_scores = sorted(scores, key=lambda score: score.total, reverse=True)

print("Scoring is done and scores are sorted")
print(len(sorted_scores))


Scoring is done and scores are sorted
1036


In [270]:
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import plotly.io as pio

import os
import numpy as np

init_notebook_mode(connected=True)


N = 0
x = np.random.rand(N)
y = np.random.rand(N)
colors = np.random.rand(N)
sz = np.random.rand(N)*30

for i in sorted_scores:
    x = np.append(x, i.item['lon'])
    y = np.append(y, i.item['lat'])
    sz = np.append(sz, 10 + (i.total / 10))
    colors = np.append(colors, i.total)


fig = go.Figure()
fig.add_scatter(x=x,
                y=y,
                mode='markers',
                marker={'size': sz,
                        'color': colors,
                        'opacity': 0.2,
                        'colorscale': 'Viridis'
                       });
iplot(fig)


In [210]:
print("..About to start: ", len(sorted_scores))
sp1 = createSuperPoint()
print("..After CREATE superPoint has: ", len(sp1.items))
print("..After CREATE sorted_scores has: ", len(sorted_scores))
nourishSuperPoint(sp1)
print("..After NOURISH superPoint has: ", len(sp1.items))
print("..After NOURISH sorted_scores has: ", len(sorted_scores))
print("..Here is what is in the SP: ", len(sorted_scores))
print(getJson(sp1.items))



..About to start:  1036
..After CREATE superPoint has:  390
..After CREATE sorted_scores has:  646
..After NOURISH superPoint has:  884
..After NOURISH sorted_scores has:  152
..Here is what is in the SP:  152
[{"id": "267", "lon": 27.9474549, "lat": -26.091599499999997, "tim": "2018-09-13T23:22:02+00:00", "amount": 0.0, "morning": false, "midday": null, "evening": true, "weekend": false}, {"id": "322", "lon": 27.947473199999997, "lat": -26.0915097, "tim": "2018-09-13T20:50:38+00:00", "amount": 0.0, "morning": false, "midday": null, "evening": true, "weekend": false}, {"id": "11040", "lon": 27.9474411, "lat": -26.0915814, "tim": "2018-08-24T19:37:13+00:00", "amount": 0.0, "morning": false, "midday": null, "evening": true, "weekend": false}, {"id": "5089", "lon": 27.947438199999997, "lat": -26.0916018, "tim": "2018-09-04T20:50:57+00:00", "amount": 0.0, "morning": false, "midday": null, "evening": true, "weekend": false}, {"id": "2967", "lon": 27.9474403, "lat": -26.091526299999998, "tim

In [211]:
# sp2 = createSuperPoint()
# print("After creating superPoint # left were: ", len(sorted_scores))
# nourishSuperPoint(sp2)
# print("After nourishing superPoint # left were: ", len(sorted_scores))
# print(getJson(sp2.items))



In [212]:
# sp3 = createSuperPoint()
# print("After creating superPoint # left were: ", len(sorted_scores))
# nourishSuperPoint(sp3)
# print("After nourishing superPoint # left were: ", len(sorted_scores))
# print(getJson(sp3.items))



In [214]:
leftOver = []
for i in sorted_scores:
    leftOver.append(i.item)
print(getJson(leftOver))

[{"id": "15144", "lon": 27.927206299999998, "lat": -26.0949256, "tim": "2018-08-17T17:44:45+00:00", "amount": 0.0, "morning": false, "midday": true, "evening": false, "weekend": false}, {"id": "6313", "lon": 27.9041286, "lat": -26.127665, "tim": "2018-09-02T10:49:33+00:00", "amount": 0.0, "morning": false, "midday": true, "evening": false, "weekend": true}, {"id": "14016", "lon": 27.9042241, "lat": -26.128173399999998, "tim": "2018-08-19T09:41:29+00:00", "amount": 0.0, "morning": false, "midday": true, "evening": false, "weekend": true}, {"id": "13278", "lon": 27.927578000000004, "lat": -26.094929099999998, "tim": "2018-08-20T17:27:01+00:00", "amount": 0.0, "morning": false, "midday": true, "evening": false, "weekend": false}, {"id": "13276", "lon": 27.9275609, "lat": -26.094925099999998, "tim": "2018-08-20T17:33:50+00:00", "amount": 0.0, "morning": false, "midday": true, "evening": false, "weekend": false}, {"id": "15152", "lon": 27.927604499999998, "lat": -26.095311900000002, "tim": 